In [2]:
import pandas as pd
import numpy as np

import time

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
import player_stats

In [4]:
positions = ['FW', 'MF', 'DF', 'GK']

## ENGLAND

In [5]:
df = pd.DataFrame()

for i in range(2020, 2022):
    stats = player_stats.player_stats("eng", i, "stats")
    stats = stats.set_index('player_id')
    playing_time = player_stats.player_stats("eng", i, "playingtime")
    playing_time = playing_time.loc[playing_time['games'] > 0]
    playing_time = playing_time.set_index('player_id')

    new_df = pd.concat([stats, playing_time], axis = 1)

    new_df['season'] = f"{str(i)}-{str(i+1)}"

    df = pd.concat([df, new_df])

df = df.loc[:,~df.columns.duplicated()]
df = df.reset_index()
df = df.loc[df['minutes_90s'] > 10]
df['position'] = df['position'].str[:2]

df['gls_value'] = round(df['goals'] * (1 + (df['goals'] / df['xg'])), 3)
df['gls_contributed'] = df['gls_value'] + df['xg_assist']

df['gls_ast'] = df['goals'] + df['assists']
df['expected'] = df['xg'] + df['xg_assist']

df['ga90'] = round(df['gls_ast'] / df['minutes_90s'], 2)
df['xga90'] = round(df['expected'] / df['minutes_90s'], 2)
df['gc90'] = round(df['gls_contributed'] / df['minutes_90s'], 2)

df = df.sort_values(by = 'gls_contributed', ascending = False)

df = df[['player_id', 'player', 'position', 'season', 'team', 'minutes_90s', 'points_per_game', 'ga90', 'xga90', 'gc90']]

df = df.dropna()

for pos in positions:

    pos_df = df.loc[df['position'] == pos]
    pos_df = pos_df.select_dtypes(include=['number'])

    if len(pos_df) >= 10:

        print(pos)

        corr_matrix = pos_df.corr()
        corr_matrix = corr_matrix.loc['points_per_game']
        print(corr_matrix)

    else: pass

FW
minutes_90s        0.087693
points_per_game    1.000000
ga90               0.543082
xga90              0.533957
gc90               0.394921
Name: points_per_game, dtype: float64
MF
minutes_90s       -0.009981
points_per_game    1.000000
ga90               0.397976
xga90              0.330946
gc90               0.335706
Name: points_per_game, dtype: float64
DF
minutes_90s       -0.040468
points_per_game    1.000000
ga90               0.313756
xga90              0.240538
gc90               0.271961
Name: points_per_game, dtype: float64
